In [1]:
import CONST
import numpy as np
import cv2
import os
import time
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras import applications
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [32]:
TRAIN_SAMPLES = CONST.FRAMES_PER_VIDEO*NUM_CLASSES*(1-TEST_SIZE) #112
VAL_SAMPLES = CONST.FRAMES_PER_VIDEO*NUM_CLASSES*TEST_SIZE #38

In [3]:
# PREPROCESSING DATASET -----------------
print("Loading dataset...")

#grab the paths to our input images followed by shuffling them 
imagePaths = sorted(list(paths.list_images('dataset')))

Loading dataset...


In [4]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('dataset'):
    if not j.startswith('.'): # If running this code locally, this is to 
        print(j)                      # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

MX4
Peace
fist


{'MX4': 0, 'Peace': 1, 'fist': 2}

In [5]:
data = []
labels = []
lookup = {}
for imagePath in imagePaths:
    # load the image, pre-process it, resize it to IMAGE_SIZE, store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (CONST.IMAGE_SIZE, CONST.IMAGE_SIZE))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    folders = []
    for folder in os.listdir('dataset'):
        if not folder.startswith('.'):
            folders.append(folder)
    for i in range(len(folders)):
        if label == folders[i]:
            label = i
            labels.append(label)
            if folders[i] not in lookup.keys():
                lookup[i]= folders[i]
            else:
                pass
    
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [6]:
# scaling the data points from [0, 255] to the range [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data,labels,test_size = 0.1)
X_train[0].shape

(100, 100, 1)

In [8]:
# convert the labels from integers to vectors
NUM_CLASSES = len(folders)

y_train = to_categorical(y_train).astype(int)
y_test = to_categorical(y_test).astype(int)

In [9]:
def save_bottlebeck_features():
    
    print("Training the VGG16 bottleneck features ...")
    
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights=None, input_shape=(100,100,1)) 
    
    bottleneck_features_train = model.predict(X_train, verbose=1)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)
    
    bottleneck_features_validation  = model.predict(X_test, verbose=1)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

In [10]:
save_bottlebeck_features()

Training the VGG16 bottleneck features ...
Instructions for updating:
Colocations handled automatically by placer.
30/30 [==============================] - 4s 137ms/step


In [11]:
print("[INFO] training the top of the model...")

[INFO] training the top of the model...


In [12]:
#save_bottlebeck_features()

train_data = np.load('bottleneck_features_train.npy')
validation_data = np.load('bottleneck_features_validation.npy')


In [22]:
from keras import layers

In [24]:
top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.6))
top_model.add(Dense(3, activation='sigmoid'))

In [25]:
top_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
EPOCHS = 50
BATCH_SIZE = 2
NUM_CLASSES = 3
TEST_SIZE = 0.25

In [35]:
start = time.time()

model_info = top_model.fit(train_data, y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,
                           validation_data=(validation_data, y_test),
                           verbose=1)
end = time.time()

print ("\nModel training time: %0.1fs\n" % (end - start))

Train on 270 samples, validate on 30 samples
Epoch 1/50
270/270 [==============================] - 2s 8ms/step - loss: 0.1909 - accuracy: 0.9704 - val_loss: 0.0834 - val_accuracy: 1.0000
Epoch 2/50
270/270 [==============================] - 2s 8ms/step - loss: 0.1899 - accuracy: 0.9630 - val_loss: 0.0794 - val_accuracy: 1.0000
Epoch 3/50
270/270 [==============================] - 2s 8ms/step - loss: 0.1747 - accuracy: 0.9630 - val_loss: 0.0766 - val_accuracy: 1.0000
Epoch 4/50
270/270 [==============================] - 2s 8ms/step - loss: 0.1849 - accuracy: 0.9667 - val_loss: 0.0692 - val_accuracy: 1.0000
Epoch 5/50
270/270 [==============================] - 2s 9ms/step - loss: 0.1822 - accuracy: 0.9741 - val_loss: 0.0651 - val_accuracy: 1.0000
Epoch 6/50
270/270 [==============================] - 2s 8ms/step - loss: 0.1885 - accuracy: 0.9444 - val_loss: 0.0607 - val_accuracy: 1.0000
Epoch 7/50
270/270 [==============================] - 2s 8ms/step - loss: 0.1859 - accuracy: 0.9667 - v

In [36]:
scores = top_model.evaluate(validation_data, y_test)
print("\nTest Loss:  %.2f%%" % (scores[0]*100))
print("Test Accuracy: %.2f%%\n" % (scores[1]*100))

30/30 [==============================] - 0s 335us/step

Test Loss:  0.86%
Test Accuracy: 100.00%



In [37]:
# Saving model
if not os.path.isdir(CONST.SAVE_DIR):
    os.makedirs(CONST.SAVE_DIR)
model_path = os.path.join(CONST.SAVE_DIR, CONST.BOTTLENECK_MODEL)
top_model.save(model_path)
print('\nSaved trained model at %s ' % model_path)


Saved trained model at /Users/Ignacio/Documents/Ironhack/datamad0819/Ejercicios/datamad0819/final-project/saved_models/bottleneck_model.h5 
